# ספר 1: יצירת וקטור מילים
בספר זה נדבר על תהליך יצרית המילים. אנו נראה איך הגענו מקובץ pdf

יש באו תמונה. לוקטור בעל מילים משמעותיות בשביל ליצור אינפות למכונה כמו שדברנו בספר הראשון

הספר זה מוכרב שני קבצי פייתון שהופכים לספר אחד

## import


In [44]:
import os #ספיריה שנותנת לנו לעשות פעולות של מערכת הפעלה
from pdf2image import convert_from_path #ספרייה שהופעת לנו pdf לתמונה
import pytesseract # ספרייה ocr של גוגל מוצאיה מילים מתמונה
import CleanText #קובץ שני של ספר 1 ניתן להתעלם ממנו בשלב זה
from collections import Counter # פעולות לאיחוד בני נתונים מורכבים
from numba import jit #ספריה שנות לנו לקופילציה למעבד
import re #  ספריה של regular expression
from num2words import num2words #תרגום מילולי של מספרים
#from googletrans import Translator
from google_trans_new import google_translator #ספריית תרגום של שפות העולם
import HebrewNlp # קובץ נוסף לספר 1
import config # דף הגדרות מערכת

## הוצאת מילים מתמונה
הפונקציה זאת מקבלת מיקום של תמונה.

הפונקציה מחזירה את המילים המופעים בתוך התמונה

In [45]:
@jit(parallel=True) #הוראת מכונה חפעלת קוד על הgpu
def imageToword(PicName):
    pytesseract.pytesseract.tesseract_cmd = config.PYTESSERACT #פעולת הפעלה של גוגל ocr
    return pytesseract.image_to_string(os.path.join(config.WORKSPACE, PicName), lang=config.PYTESSERACT_LANGUAGE)


## פעולת הוצאת תמונה מpdf
הפעולה מקבלת כתובת של הקובץ, שם קובץ,משתני בולייני אם התמונה נועדה לחתוך

הפעולה מחזירה מיקום של קובץ של תמונה או את המלים שבתוכו טלוי במשתנה הבוליאני

In [46]:
def ReadPdf(root,name,iscrop):
    pages = []
    pages = convert_from_path(root, poppler_path=os.path.join(config.POPPLER))
    (Name,file) = name.split('.')
    PicName =Name+"."+config.IMAGETYPE
    pages[0].save(os.path.join(config.WORKSPACE, PicName), config.IMAGETYPE)
    if iscrop: 
        return os.path.join(config.WORKSPACE, PicName) #המטרה של החלק הזה היא בשביל חלק אחר של הפרויקט נדבר על זה בהמשך
    else:
        words = imageToword(PicName)
        os.remove(os.path.join(config.WORKSPACE, PicName)) #אחרי קבלת המילים נמחק את התמונה בשביל לא לתפוס מקום
        return words

## ניקוי המילים 
אחרי שהתאנו את המילים המקובץ אנחנו רוצים להפתר מצרופים לא הגיונים אשר מורידים את יעילות המכונה

In [47]:
#הפעולה מקבלת טקסט גולמי 
#הפעולה מחזירה את הטקסט נקי
def CleanTheText(Text):
    Text =" ".join(Text.split())
    Text = Translate2Word(Text)
    Text = CleanRound1(Text)
    return (Text)

#הפעולה מקבלת טקסט עם מספרים ואתיות באנגלית 
#המכונה מתרגמת את המספרים והאותיות באנגלית לשפה בנחרת
def Translate2Word(text):
 #   translator = Translator(['translate.googleapis.com','translate.google.com','translate.google.co.kr'])
    translator = google_translator() 
    wordsarr =text.split(' ')
    for i, element in enumerate(wordsarr):# לולאה אשר בודקת את כל המילים בטקסט
        if(element.isnumeric()): #בודקת שזה מספר ותרגמת אותו
            text2 = num2words(int(element), lang='en')
            #text = translator.translate(text2, dest='he').text
            text = translator.translate(text2,lang_tgt=config.SYSTEMLANGUAGE)
            wordsarr[i] = text
    text = " ".join(wordsarr)
    return text

#הפעולה מקבלת טקסט גולמי
#הפעולה מחזירה טקסט נקיי ללא לסמנים מיוחדים או רווחים מיותרים
def CleanRound1(Text):
    Text = re.sub('[…]', '', Text) #נקיו סוגרים
    Text = re.sub('[%s]' % re.escape(r'!#$%&*₪+\/,©-():;<=>[]^_-־`?{|}~."\''), '', Text) # נקוי סמנים מיוחדים
    Text = ' '.join([w for w in Text.split() if len(w)>1] )
    Text = re.sub('[ץףםןך^]', ' ', Text) # מחיקת מלים אשר מתחילות או באמצע המילה יש אותיות ספיות
    Text = re.sub('   ', ' ', Text)
    Text = re.sub('  ', ' ', Text)
    Text = re.sub('\d+', '', Text) #מחיקת אנטרים
    return Text

## Tokenizer
הפקונרציה זאת לוקחת את כל המילים במערך ומשנה את המילים למילים של השורש למשל מהנדסים מהנדסות הופכים ל מנהדס

In [48]:
def HebrewTokenizer(Text):
    try:
        request = {"token":  config.TOKENHEBNLP ,"text": Text} #קריאה לapi חיצוני שיעשה את זה
        return requests.post(config.TOKENIZERURL, json=request).json()[0] 
    except:
        arry = Text.split(' ')
        return  arry

## שימשו כל הפונקציות
הפעולה מקבלת כתובת של קובץ ושם קובץ

הפעולה מחזירה וקטור של מידע נקי של מילים. 

In [49]:
def Tokenize(root,name):
    fileName = ReadPdf(root,name,False)
    text = CleanText.CleanTheText(fileName)

    tokenizer = HebrewNlp.HebrewTokenizer(text)
    for i, element in enumerate(tokenizer): #לולרב בודקת עם המילה יותר מאות אחת
        if (len(element) <= 1):
            tokenizer.pop(i)
    return tokenizer

## פקנציות נספות שהספר הזה יכול לעשות 

In [50]:
#הפעולה מקבלת ווקטור מילין,מערך של כל שמות הפרטים, מערך של כל שמות המישפחה 
#הפונקציה מחזירה שם  מלא בכתוב בווקטור
def GetName(text,allname,allfname):
    if(type(text)== str):
        arry = text.split(" ")
    else:
        arry = text
    i= 0
    firstname = ""
    lastname = ""
    havefullname = False
    while i < (len(arry)) and not havefullname: ##כל החיפושים הם חפושים בנארים לכן יעלות הפונקציה הזאת היא nlog(n)
        if lastname == "" and not (i+1==len(arry)):
            two=arry[i]+ " "+arry[i+1]
            if(two in allfname ):
                lastname = two
                i+=1
            elif str(arry[i]) in allfname:
                lastname = arry[i]
        elif(firstname == "" and  i+2 <len(arry)):
            two = arry[i+1]+ " "+ arry[i+2]
            if(str(arry[i]) in allname and two != "שם האב."):
                    firstname = arry[i]
        if (not(firstname == "") and not (lastname=="")):
            havefullname = True
        i+=1
    if havefullname:
        return firstname + " "+ lastname
    else:
        return config.ERRORLABEL
    
  #הפונקציה זאת מקבלת תמיקום של תמונה מערך של כל שמות המישפחה ומחזירה את השם המלא של אדם
def GetName(PicName,allname,allfname):
    text = imageToword(PicName)
    new_text  =" ".join(text.split())
    return HebrewNlp.GetName(new_text,allname,allfname)

# הפעולה מקבלת שני מילונים 
#הפעולה מאחדת את שני המלונים השונים
def margedictionaries(dictionarie1,dictionarie2):
    ini_dictionary1 = Counter(dictionarie1)
    ini_dictionary2 = Counter(dictionarie2)
    return ini_dictionary1 + ini_dictionary2

##  בניית הדטה בשביל אימון המוכנה
אחרי שהצלחנו לזהות את המילים ונקינו אותם אנחנו רוצים לבנות רשימה של כל הנתונים החשובים

לכן יצרנו פוקציה אשר בונה את המידע החושב 

In [51]:
def BuildModelToTraing(root,name):
    #הפונקציה מקבלת שם קובץ ומיקום קובץ
    #הפונקציה מחזירה רישמה של סוג מסמך,מיקומו ,והמילים שיש בפנים
    #מחזירה מילון שיכולות מילים
    rap = ()
    #try:
    if 1==1:
        wordfreq = {}
        wordtoken = Tokenize(root,name)
        for token in wordtoken: #לולא בונא את מילון שכותבת כמה הופעות יש לכל מילה
            if token not in wordfreq.keys():
                wordfreq[token] = 1
            else:
                wordfreq[token] += 1
        rap = [config.ERRORLABEL, root, wordtoken] 
        if config.REPORTTYPE1 in root:           #באימון המכונה אז בכתוב היה כתוב איפה ניצא הקובץ
            rap = [config.REPORTTYPE1, root,wordtoken]
        if config.REPORTTYPE2 in root:       #באימון המכונה אז בכתוב היה כתוב איפה ניצא הקובץ
            rap = [config.REPORTTYPE2,root, wordtoken] #לקובץ חדש זה יכתוב לבדיקה
        return [rap,wordfreq]
 #   except Exception as e:
     #   return [config.ERRORLABEL,root,config.ERRORLABEL]


## תוצאה של כל התהליכים יחד
הפעולה יכולה לקחת כמה דקות. 

In [52]:
import warnings
warnings.filterwarnings("ignore")
print(BuildModelToTraing(os.path.join("exmple","2015.pdf"),"2015.pdf"))

[['לבדיקה', 'exmple\\2015.pdf', ['טופס', 'אל', 'שלוש', 'מאות', 'ואחד', 'שודר', 'עי', 'מייצג', 'באינטראנט', 'אחד', 'מתו', 'ארבעה', 'התשמיוס', 'ליחיד', 'וו', 'וו', 'אסמכתא', 'חותמת', 'המשרד', 'הרשו', 'בלשו', 'זכר', 'מתיחס', 'לנקבה', 'די', 'וחשבו', 'על', 'ההכנסות', 'באר', 'ובחול', 'תשע', 'תשע', 'תארי', 'הגשת', 'הדוח', 'ור', 'הדוח', 'בשנת', 'המס', 'אלפיי', 'ושמונהעשרה', 'שתיי', 'השנה', 'המתחילה', 'והמסתיימת', 'השומה', 'תל', 'אביב', 'חמש', 'חוליה', 'שבעה', 'עפי', 'סעי', 'מאה', 'ושלושי', 'ואחת', 'לפקודת', 'מס', 'הכנסה', 'רשאי', 'פקיד', 'השומה', 'לראות', 'מי', 'שהגיש', 'דוח', 'שלא', 'כלליי', 'מולא', 'כראוי', 'או', 'שלא', 'צורפו', 'אליו', 'המסמכי', 'המתאימי', 'כמי', 'שלא', 'הגיש', 'דוח', 'על', 'הכנסותי', 'והכנסות', 'בת', 'זוגי', 'שלושה', 'הכנסותי', 'בלבד', 'חמש', 'אני', 'מגיש', 'דוח', 'לשנת', 'מס', 'זו', 'למרות', 'שאיני', 'חייבבקשה', 'להחוזר', 'מס', 'וגי', 'מגיש', 'דוח', 'נפרד', 'מצב', 'הדוחהצהרה', 'של', 'בת', 'זוגי', 'אי', 'הכנסה', 'לב', 'בת', 'זוגי', 'בת', 'זוגי', 'עזר', 'לי', 'בהשגת', 'הכנס